In [22]:
!pip freeze > ../requrements.txt

In [1]:
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, sys
import fastcore.all as fc
from pathlib import Path
import numpy as np, matplotlib.pyplot as plt
from collections.abc import Mapping, Callable, Iterable
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial, wraps
import copy
from contextlib import contextmanager

import torch
from torch import tensor, nn, optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms.functional as TF
from torcheval.metrics import MulticlassAccuracy, Mean
from datasets import load_dataset, load_dataset_builder
import logging

from fastprogress import progress_bar, master_bar

sys.path.append(os.path.abspath('..'))

c:\Users\KORALLLL\Desktop\mtuci\diffusion\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'
def_device = 'cpu'

logging.disable(logging.WARNING)

In [3]:
from utils import inplace_windows
from data_utils import collate_dict

In [4]:
x, y = 'image', 'label'
name = "fashion_mnist"
dataset = load_dataset(name, trust_remote_code=True)

@inplace_windows
def transformi(b):
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
    
batch_size = 1024

tranformed_dataset = dataset.with_transform(transformi)

c:\Users\KORALLLL\Desktop\mtuci\diffusion\utils\inplace.py:21: UserWarning: Inplace doesn't work on windows
  warnings.warn("Inplace doesn't work on windows")


In [5]:
class DataLoaders:
    def __init__(self, *dataloaders):
        self.train, self.valid = dataloaders[:2]
    
    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True):
        """
        from dataset dict
        """
        return cls(*[DataLoader(ds, batch_size, collate_fn=collate_dict(ds), drop_last=True, num_workers=2)
                    for ds in dd.values()])

In [6]:
dls = DataLoaders.from_dd(tranformed_dataset, batch_size)
train_loader = dls.train
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

AttributeError: Can't pickle local object 'collate_dict.<locals>._f'